# Install Dependencies

In [4]:
!pip install pyspark

# Read Json

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("COMP5349 A2 Data Loading") \
    .getOrCreate()

data = "Assignment_2_data/test.json"
init_df = spark.read.json(data)

In [6]:
from pyspark.sql.functions import explode
from pyspark.sql.functions import udf

data_df= init_df.select((explode("data").alias('data')))
paragraph_df = data_df.select(explode("data.paragraphs").alias("paragraph"))

# data_df.printSchema()
# paragraph_df.printSchema()
# paragraph_df.show(5)

In [7]:
paragraph_unrolled_df = paragraph_df.select("paragraph.context" , explode("paragraph.qas").alias("qas")) \
                                    .select("context", "qas.id", "qas.question", "qas.is_impossible", explode("qas.answers").alias("answer")) \
                                    .select("context", "id", "question", "is_impossible", "answer.answer_start", "answer.text")

# paragraph_unrolled_df.show(5)
# paragraph_unrolled_df.printSchema()

In [8]:
import re
from pyspark.sql.types import ArrayType, StringType

@udf(returnType = ArrayType(StringType()))
def segmentContext(str):
    res = [str[i:i+512] for i in range(0, len(str)-256, 256)]
    return res

# res = segmentContext("1111122222\n\n\n\n\n333334444455")
# print(res)

paragraph_preproc_df = paragraph_unrolled_df.withColumn("sequence", explode(segmentContext("context"))).drop("context")

In [ ]:
positive_sample_df = paragraph_preproc_df.filter("is_impossible=False")
positive_sample_df.show(5)